# How to listen to events, and upload model's output on chain

By calling contract!

In [39]:
from web3 import Web3, HTTPProvider
from hexbytes import HexBytes
import json
import time
import os
from threading import Thread
from web3.datastructures import AttributeDict

In [37]:
w3 = Web3(HTTPProvider(''))
my_address = ""
private_key = ""
address = Web3.to_checksum_address(my_address)

In [41]:
collector_contract_address = "0x7f8A0D82d1f14D0976bE8AaAb0C885e3Be154c43"
collector_contract_abi = json.loads('[{"inputs":[{"internalType":"address","name":"_address","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"address","name":"owner","type":"address"}],"name":"OwnableInvalidOwner","type":"error"},{"inputs":[{"internalType":"address","name":"account","type":"address"}],"name":"OwnableUnauthorizedAccount","type":"error"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"previousOwner","type":"address"},{"indexed":true,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"requestId","type":"uint256"},{"indexed":false,"internalType":"string","name":"output","type":"string"}],"name":"ResponseEvent","type":"event"},{"inputs":[],"name":"coordinator","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_id","type":"uint256"}],"name":"getOutput","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"","type":"uint256"}],"name":"outputs","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"renounceOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_coordinator","type":"address"}],"name":"setCoordinator","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_id","type":"uint256"},{"internalType":"string","name":"_output","type":"string"}],"name":"setOutput","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"}]')
collector_contract = w3.eth.contract(address=collector_contract_address, abi=collector_contract_abi)

coordinator_contract_address = "0x1B42d9Ba11180Db82C5fd902a40D769987fF1c3B"
coordinator_contract_abi = json.loads('[{"inputs": [],"stateMutability": "nonpayable","type": "constructor"},{"anonymous": false,"inputs": [{"indexed": false,"internalType": "uint256","name": "requestId","type": "uint256"},{"indexed": false,"internalType": "address","name": "user","type": "address"},{"indexed": false,"internalType": "string","name": "modelId","type": "string"},{"indexed": false,"internalType": "string","name": "input","type": "string"},{"indexed": false,"internalType": "address","name": "callbackContract","type": "address"},{"indexed": false,"internalType": "string","name": "callbackFunction","type": "string"}],"name": "RequestEvent","type": "event"},{"anonymous": false,"inputs": [{"indexed": false,"internalType": "uint256","name": "requestId","type": "uint256"},{"indexed": false,"internalType": "string","name": "output","type": "string"}],"name": "ResponseEvent","type": "event"},{"inputs": [{"internalType": "uint256","name": "_id","type": "uint256"},{"internalType": "string","name": "_output","type": "string"}],"name": "fulfillRequest","outputs": [],"stateMutability": "nonpayable","type": "function"},{"inputs": [{"internalType": "uint256","name": "_id","type": "uint256"}],"name": "getOutput","outputs": [{"internalType": "string","name": "","type": "string"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "uint256","name": "_id","type": "uint256"}],"name": "getRequest","outputs": [{"internalType": "address","name": "","type": "address"},{"internalType": "string","name": "","type": "string"},{"internalType": "string","name": "","type": "string"},{"internalType": "address","name": "","type": "address"},{"internalType": "string","name": "","type": "string"},{"internalType": "bool","name": "","type": "bool"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "string","name": "_modelId","type": "string"},{"internalType": "string","name": "_input","type": "string"},{"internalType": "address","name": "_callbackContract","type": "address"},{"internalType": "string","name": "_callbackFunction","type": "string"}],"name": "makeRequest","outputs": [],"stateMutability": "nonpayable","type": "function"},{"inputs": [{"internalType": "uint256","name": "","type": "uint256"}],"name": "outputs","outputs": [{"internalType": "string","name": "","type": "string"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "uint256","name": "","type": "uint256"}],"name": "requestStatus","outputs": [{"internalType": "bool","name": "","type": "bool"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "uint256","name": "","type": "uint256"}],"name": "requests","outputs": [{"internalType": "address","name": "user","type": "address"},{"internalType": "string","name": "modelId","type": "string"},{"internalType": "string","name": "input","type": "string"},{"internalType": "address","name": "callbackContract","type": "address"},{"internalType": "string","name": "callbackFunction","type": "string"}],"stateMutability": "view","type": "function"}]')
coordinator_contract = w3.eth.contract(address=coordinator_contract_address, abi=coordinator_contract_abi)

In [46]:
request_event_filter = coordinator_contract.events.RequestEvent.create_filter(fromBlock='latest')

# You can use your model here
def my_model(input, modelId = 1234):
    return input + " processed"

stop_thread = False

def handle_event(event):
    print("="*10)
    # Parse the event
    args = event['args']
    requestId = args['requestId']
    user = args['user']
    modelId = args['modelId']
    input = args['input']
    callbackContract = args['callbackContract']
    callbackFunction = args['callbackFunction']
    print(f"Request ID: {requestId}")
    print(f"Received request from {user} for model {modelId} with input {input}")
    print(f"Callback contract: {callbackContract}")
    print(f"Callback function: {callbackFunction}")

    # Process the input, get the output using your model with the modelId
    output = my_model(input, modelId)
    print(f"\nOutput: {output}")

    # Send the output back to the coordinator to fulfill the request
    tx_hash = coordinator_contract.functions.fulfillRequest(requestId, output).build_transaction({'from': address,"nonce": w3.eth.get_transaction_count(address), "gas": 1000000, "gasPrice": w3.eth.gas_price, "chainId": w3.eth.chain_id, "value": 0})
    signed_tx = w3.eth.account.sign_transaction(tx_hash, private_key=private_key)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
    print(f"Transaction hash: {tx_hash.hex()}")

def log_loop(event_filter, poll_interval):
    while not stop_thread:
        for event in event_filter.get_new_entries():
            handle_event(event)
        time.sleep(poll_interval)

worker = Thread(target=log_loop, args=(request_event_filter, 2), daemon=True)
worker.start()
time.sleep(200)
stop_thread = True
worker.join()
print("="*10)
print("finished!")

Request ID: 61449662441728798945671122236511050868222596957928297598899945623039085479140
Received request from 0x3DAD7c5704A6eA567A0549ee711768C0A405D06C for model 11111 with input TestInput1234
Callback contract: 0x7f8A0D82d1f14D0976bE8AaAb0C885e3Be154c43
Callback function: setOutput(uint256,string)

Output: TestInput1234 processed
Transaction hash: 0x0032fb06380dfbe1902322efc1c280de0ec3ef0deea15e7fda76413fc4453a75
Request ID: 93167671792678172914998233092221646823159126762632369763689914728220711443173
Received request from 0x3DAD7c5704A6eA567A0549ee711768C0A405D06C for model 11111 with input TestInput5678
Callback contract: 0x7f8A0D82d1f14D0976bE8AaAb0C885e3Be154c43
Callback function: setOutput(uint256,string)

Output: TestInput5678 processed
Transaction hash: 0x79bcb3b801bfc899bc7143de95bc684144b347174b77c881ae13ccc08ecd1380
Request ID: 42080307265764545085978107958747233995913156732739041136131800900114180654393
Received request from 0x3DAD7c5704A6eA567A0549ee711768C0A405D06C for

#### After you fulfill the request, you can use the Request ID to check the output in [this contract](https://sepolia.etherscan.io/address/0x7f8A0D82d1f14D0976bE8AaAb0C885e3Be154c43#readContract)